# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 20 stocks with the highest price momentum making use of a number of additional criteria as suggested by Tal Davidson: https://taldavidson.com/quantitative-momentum/quantitative-momentum-strategy/. 

From there, we will calculate recommended trades for an equal-weight portfolio of these 20 stocks. 

Due to the constraints of the available APIs, we will focus exclusively on US-based stocks, specifically those listed on the S&P 500 index.

### Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this project.

In [1]:
from forex_python.converter import CurrencyRates
import math
import numpy as np
import os
import pandas as pd
import pickle
import pyEX
import requests
from scipy.stats import percentileofscore as score
import xlsxwriter

### Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500. \
In this case we are going to use a static version of it but feel free to choose a different approach.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


### Acquiring API-Token

Now it's time to import our IEX Cloud API token. This is the data provider that I will be using throughout this project.

My key is being stored in an environment variable here, so you need to provide your own if you want to replicate this code.


In [3]:
IEX_CLOUD_API_TOKEN = os.environ.get("IEX_SANDBOX_KEY")

# Step 1 - Making our First API-Call

Now it's time to structure our API calls to IEX cloud.

An example of a simple request could look like this:

In [4]:
symbol = 'LB'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'L Brands Inc',
 'marketcap': 22722212378,
 'week52high': 84,
 'week52low': 25.74,
 'week52highSplitAdjustOnly': 85,
 'week52lowSplitAdjustOnly': 25.68,
 'week52change': 2.343094435246564,
 'sharesOutstanding': 281210025,
 'float': 0,
 'avg10Volume': 3250980,
 'avg30Volume': 4887293,
 'day200MovingAvg': 63.03,
 'day50MovingAvg': 74.54,
 'employees': 58244,
 'ttmEPS': 5.14,
 'ttmDividendRate': 0.15338359512469846,
 'dividendYield': 0.001945140892978844,
 'nextDividendDate': '',
 'exDividendDate': '2021-05-26',
 'nextEarningsDate': '',
 'peRatio': 16.00346153055435,
 'beta': 1.3635047776186482,
 'maxChangePercent': 3.12240148472899,
 'year5ChangePercent': 0.3722093592469524,
 'year2ChangePercent': 2.477061854566112,
 'year1ChangePercent': 2.337482713787379,
 'ytdChangePercent': 1.173918687686791,
 'month6ChangePercent': 0.834489469878399,
 'month3ChangePercent': 0.22102352924366842,
 'month1ChangePercent': 0.08282482196304282,
 'day30ChangePercent': 0.08145367741338766,
 

### Executing A Batch API Call & Building Our DataFrame

The first thing we need to do is to add the following data to a pandas DataFrame:

- The current price of each stock
- One-Year price return for each stock
- Six-Month price return for each stock
- Three-Month price return for each stock
- One-Month price return for each stock
- Beta of each stock

Batch API calls are one of the easiest ways to improve the performance of our code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often offer discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

We'll start by running the following code cell, which contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker',
                'Price', 
                'One-Year Price Return', 
                'One-Month Price Return',
                'Beta', 
                'One-Year IM', 
                'One-Year IM-Percentile', 
                'Six-Month Price Return', 
                'Six-Month IM', 
                'Six-Month IM-Percentile',
                'Three-Month Price Return', 
                'Three-Month IM', 
                'Three-Month IM-Percentile', 
                'Piotroski Score', 
                'ROC',
                'ROC Percentile', 
                'Number of Shares to Buy']

Now we can create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
momentum_df = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        momentum_df = momentum_df.append(
                                        pd.Series([symbol,
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   data[symbol]['stats']['beta'],
                                                   'N/A',
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   'N/A',
                                                   'N/A',
                                                   'N/A',
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
## Run this to work around error in percentileofscore function
momentum_df['One-Year Price Return'] = momentum_df['One-Year Price Return'].astype('float')
momentum_df['Six-Month Price Return'] = momentum_df['Six-Month Price Return'].astype('float')
momentum_df['Three-Month Price Return'] = momentum_df['Three-Month Price Return'].astype('float')
momentum_df['One-Month Price Return'] = momentum_df['One-Month Price Return'].astype('float')

momentum_df

,Ticker,Price,One-Year Price Return,One-Month Price Return,Beta,One-Year IM,One-Year IM-Percentile,Six-Month Price Return,Six-Month IM,Six-Month IM-Percentile,Three-Month Price Return,Three-Month IM,Three-Month IM-Percentile,Piotroski Score,ROC,ROC Percentile,Number of Shares to Buy
0,A,168.87,0.731753,0.140494,0.947536,N/A,N/A,0.329178,N/A,N/A,0.296226,N/A,N/A,N/A,N/A,N/A,N/A
1,AAL,19.30,0.488289,-0.012698,1.154142,N/A,N/A,0.002749,N/A,N/A,-0.187446,N/A,N/A,N/A,N/A,N/A,N/A
2,AAP,215.62,0.317672,0.013635,0.714996,N/A,N/A,0.265575,N/A,N/A,0.078741,N/A,N/A,N/A,N/A,N/A,N/A
3,AAPL,148.81,0.284063,0.031956,1.577257,N/A,N/A,0.136210,N/A,N/A,0.183090,N/A,N/A,N/A,N/A,N/A,N/A
4,ABBV,119.25,0.293826,0.022279,0.686594,N/A,N/A,0.149568,N/A,N/A,0.029593,N/A,N/A,N/A,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,135.16,0.474020,0.182510,0.701742,N/A,N/A,0.283499,N/A,N/A,0.135743,N/A,N/A,N/A,N/A,N/A,N/A
501,ZBH,149.76,0.058961,-0.041849,0.967284,N/A,N/A,-0.078482,N/A,N/A,-0.132610,N/A,N/A,N/A,N/A,N/A,N/A
502,ZBRA,591.28,1.000159,0.115488,1.327935,N/A,N/A,0.152769,N/A,N/A,0.167733,N/A,N/A,N/A,N/A,N/A,N/A
503,ZION,53.99,0.667300,0.123987,1.275753,N/A,N/A,0.041602,N/A,N/A,-0.070784,N/A,N/A,N/A,N/A,N/A,N/A


### Exluding Most Recent Month From Annual Price Return

The practice of excluding the most recent month in the calculation of momentum is consistent with the academic research showing that month-to-month stock performance tends to invert. Previous month losers tend to be winners in the following month. Previous month winners tend to be losers in the next month.

We can obtain the momentum score without the most recent month by simply substracting the One-Month Price Returns from the One-Year ones.

In [7]:
excl_last_month = momentum_df['One-Year Price Return'] - momentum_df['One-Month Price Return']
momentum_df['One-Year Price Return'] = excl_last_month
momentum_df.drop(['One-Month Price Return'], axis = 1, inplace = True)
momentum_df

,Ticker,Price,One-Year Price Return,Beta,One-Year IM,One-Year IM-Percentile,Six-Month Price Return,Six-Month IM,Six-Month IM-Percentile,Three-Month Price Return,Three-Month IM,Three-Month IM-Percentile,Piotroski Score,ROC,ROC Percentile,Number of Shares to Buy
0,A,168.87,0.591259,0.947536,N/A,N/A,0.329178,N/A,N/A,0.296226,N/A,N/A,N/A,N/A,N/A,N/A
1,AAL,19.30,0.500988,1.154142,N/A,N/A,0.002749,N/A,N/A,-0.187446,N/A,N/A,N/A,N/A,N/A,N/A
2,AAP,215.62,0.304037,0.714996,N/A,N/A,0.265575,N/A,N/A,0.078741,N/A,N/A,N/A,N/A,N/A,N/A
3,AAPL,148.81,0.252107,1.577257,N/A,N/A,0.136210,N/A,N/A,0.183090,N/A,N/A,N/A,N/A,N/A,N/A
4,ABBV,119.25,0.271548,0.686594,N/A,N/A,0.149568,N/A,N/A,0.029593,N/A,N/A,N/A,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,135.16,0.291510,0.701742,N/A,N/A,0.283499,N/A,N/A,0.135743,N/A,N/A,N/A,N/A,N/A,N/A
501,ZBH,149.76,0.100811,0.967284,N/A,N/A,-0.078482,N/A,N/A,-0.132610,N/A,N/A,N/A,N/A,N/A,N/A
502,ZBRA,591.28,0.884672,1.327935,N/A,N/A,0.152769,N/A,N/A,0.167733,N/A,N/A,N/A,N/A,N/A,N/A
503,ZION,53.99,0.543313,1.275753,N/A,N/A,0.041602,N/A,N/A,-0.070784,N/A,N/A,N/A,N/A,N/A,N/A


# Step 2 - The Idiosyncratic Momentum Criterion

Now that we have defined the base structure of our project (in Step One), it is time to further tweak our momentum selection criteria.

For this we need the following information from the API:

- One-Year price return for each stock
- Beta for each stock
- One-Year price return for the S&P 500 index

Beta is a numeric value that measures the fluctuations of a stock to changes in the overall stock market.

This enables us to normalize each stock's momentum to a common volatility baseline, since Beta is a numeric value that measures the fluctuations of a stock to changes in the overall stock market. Any momentum that remains after normalization is resulting from a stock’s unique situation, therefore regarded as idiosyncratic momentum.

The formula for idiosyncratic momentum (IM) would thus be as follows:

\begin{equation}
\text{IM} = \text{year1changeP} - (\text{beta} * \text{S&P500year1changeP})
\end{equation}

The main inspiration for this step comes from the following article: https://alphaarchitect.com/2017/05/02/swedroe-spotlight-enhancing-momentum-strategies-via-idiosyncratic-momentum/.

### Acquiring the data

Unfortunately, IEXCloud doesn't provide data for the S&P 500 index as a whole. 

In an ideal world we would thus connect directly to the index provider (Standard & Poor's) and pull the data we need on a regular basis.

However, paying for access to the index provider's API is outside the scope of this project.

What we are going to do instead, is to forgo the idea  of using the One-Year price return for the S&P 500 index and instead use the One-Year Price return for the SPDR S&P 500 ETF Trust (https://www.ssga.com/us/en/institutional/etfs/funds/spdr-sp-500-etf-trust-spy).

The SPDR S&P 500 ETF Trust (SPY) is perhaps the best-recognized and oldest US listed ETF and typically tops rankings for largest AUM (Assets under Management) and greatest trading volume. The fund tracks the massively popular US index, the S&P 500. As a UIT (Unit Investment Trust), SPY must fully replicate its index.

Thus, the SPDR S&P 500 ETF Trust closely mirrors its index (the S&P 500) and should therefore work equally well as a common volatility baseline for our stocks.

The following code returns the One-Year price return for the SPDR S&P 500 ETF Trust:

In [8]:
symbol = 'SPY'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
SPYyear1changeP = data['year1ChangePercent']
SPYmonth6changeP = data['month6ChangePercent']
SPYmonth3changeP = data['month3ChangePercent']
print(f'The One-Year Price Return of the SPDR S&P 500 ETF Trust was {SPYyear1changeP}.')
print(f'The Six-Month Price Return of the SPDR S&P 500 ETF Trust was {SPYmonth6changeP}.')
print(f'The Three-Month Price Return of the SPDR S&P 500 ETF Trust was {SPYmonth3changeP}.')

The One-Year Price Return of the SPDR S&P 500 ETF Trust was 0.3337514220681346.
The Six-Month Price Return of the SPDR S&P 500 ETF Trust was 0.1367447064686904.
The Three-Month Price Return of the SPDR S&P 500 ETF Trust was 0.07476076363384082.


### Calculating one-year idiosyncratic momentum

We are now in a position where we have got everything we need in order to calculate our Idiosyncratic Momentum, using the formula that we have defined above. We can instantly see a major difference, as the Idiosyncratic Momentum seems to be lower than the simple momentum given by the One-Year Price Return column.

In fact, some stocks now actually have negative Momentum!

In [9]:
IM = momentum_df['One-Year Price Return'] - (momentum_df['Beta'] * SPYyear1changeP)

for i in momentum_df.index:
    momentum_df.loc[i, 'One-Year IM'] = IM[i]
    
momentum_df

,Ticker,Price,One-Year Price Return,Beta,One-Year IM,One-Year IM-Percentile,Six-Month Price Return,Six-Month IM,Six-Month IM-Percentile,Three-Month Price Return,Three-Month IM,Three-Month IM-Percentile,Piotroski Score,ROC,ROC Percentile,Number of Shares to Buy
0,A,168.87,0.591259,0.947536,0.275018,N/A,0.329178,N/A,N/A,0.296226,N/A,N/A,N/A,N/A,N/A,N/A
1,AAL,19.30,0.500988,1.154142,0.115791,N/A,0.002749,N/A,N/A,-0.187446,N/A,N/A,N/A,N/A,N/A,N/A
2,AAP,215.62,0.304037,0.714996,0.065406,N/A,0.265575,N/A,N/A,0.078741,N/A,N/A,N/A,N/A,N/A,N/A
3,AAPL,148.81,0.252107,1.577257,-0.274305,N/A,0.136210,N/A,N/A,0.183090,N/A,N/A,N/A,N/A,N/A,N/A
4,ABBV,119.25,0.271548,0.686594,0.042396,N/A,0.149568,N/A,N/A,0.029593,N/A,N/A,N/A,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,135.16,0.291510,0.701742,0.057303,N/A,0.283499,N/A,N/A,0.135743,N/A,N/A,N/A,N/A,N/A,N/A
501,ZBH,149.76,0.100811,0.967284,-0.222022,N/A,-0.078482,N/A,N/A,-0.132610,N/A,N/A,N/A,N/A,N/A,N/A
502,ZBRA,591.28,0.884672,1.327935,0.441471,N/A,0.152769,N/A,N/A,0.167733,N/A,N/A,N/A,N/A,N/A,N/A
503,ZION,53.99,0.543313,1.275753,0.117529,N/A,0.041602,N/A,N/A,-0.070784,N/A,N/A,N/A,N/A,N/A,N/A


We can do the same for the six- and three month periods:

In [10]:
IM_six_month = momentum_df['Six-Month Price Return'] - (momentum_df['Beta'] * SPYmonth6changeP)

for i in momentum_df.index:
    momentum_df.loc[i, 'Six-Month IM'] = IM_six_month[i]
    
IM_three_month = momentum_df['Three-Month Price Return'] - (momentum_df['Beta'] * SPYmonth3changeP)

for i in momentum_df.index:
    momentum_df.loc[i, 'Three-Month IM'] = IM_three_month[i]
    
momentum_df

,Ticker,Price,One-Year Price Return,Beta,One-Year IM,One-Year IM-Percentile,Six-Month Price Return,Six-Month IM,Six-Month IM-Percentile,Three-Month Price Return,Three-Month IM,Three-Month IM-Percentile,Piotroski Score,ROC,ROC Percentile,Number of Shares to Buy
0,A,168.87,0.591259,0.947536,0.275018,N/A,0.329178,0.199607,N/A,0.296226,0.225388,N/A,N/A,N/A,N/A,N/A
1,AAL,19.30,0.500988,1.154142,0.115791,N/A,0.002749,-0.155073,N/A,-0.187446,-0.27373,N/A,N/A,N/A,N/A,N/A
2,AAP,215.62,0.304037,0.714996,0.065406,N/A,0.265575,0.167803,N/A,0.078741,0.025287,N/A,N/A,N/A,N/A,N/A
3,AAPL,148.81,0.252107,1.577257,-0.274305,N/A,0.136210,-0.079472,N/A,0.183090,0.065173,N/A,N/A,N/A,N/A,N/A
4,ABBV,119.25,0.271548,0.686594,0.042396,N/A,0.149568,0.05568,N/A,0.029593,-0.021737,N/A,N/A,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,135.16,0.291510,0.701742,0.057303,N/A,0.283499,0.18754,N/A,0.135743,0.08328,N/A,N/A,N/A,N/A,N/A
501,ZBH,149.76,0.100811,0.967284,-0.222022,N/A,-0.078482,-0.210752,N/A,-0.132610,-0.204925,N/A,N/A,N/A,N/A,N/A
502,ZBRA,591.28,0.884672,1.327935,0.441471,N/A,0.152769,-0.028819,N/A,0.167733,0.068455,N/A,N/A,N/A,N/A,N/A
503,ZION,53.99,0.543313,1.275753,0.117529,N/A,0.041602,-0.13285,N/A,-0.070784,-0.16616,N/A,N/A,N/A,N/A,N/A


# Step 3 - Momentum Quality

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

High-quality momentum stocks show "slow and steady" outperformance over long periods of time
Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.
The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be caused by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest two deciles of momentum stocks.

In addition, we will only consider those stocks that also rank in the upper half of stocks with highest _quarterly_ and _bi-quarterly_ idiosyncratic momentum.

We thus need to calculate momentum percentile scores for every stock in the S&P500. More specifically, we need to calculate percentile scores for the following metrics for every stock:

- (One-Year) Idiosyncratic Momentum
- Six-Month Idiosyncratic Momentum
- Three-Month Idiosyncratic Momentum

Here's how we can do this:

In [11]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month'
                ]

# Some data from the API might have a value of None, which will cause an error with the percentileofscore function.
# We thus work around this by manually assigning all 'None' values to zero.
for row in momentum_df.index:
    for time_period in time_periods:
        change_col = f'{time_period} IM'
        percentile_col = f'{time_period} IM-Percentile'
        if momentum_df.loc[row, change_col] == None:
            momentum_df.loc[row, change_col] = 0.0

for row in momentum_df.index:
    for time_period in time_periods:        
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} IM-Percentile'            
        momentum_df.loc[row, percentile_col] = score(momentum_df[change_col], momentum_df.loc[row, change_col]) / 100
        
momentum_df

C:\Users\Max\Anaconda3\lib\site-packages\scipy\stats\stats.py:1857: RuntimeWarning: invalid value encountered in less
  left = np.count_nonzero(a < score)
C:\Users\Max\Anaconda3\lib\site-packages\scipy\stats\stats.py:1858: RuntimeWarning: invalid value encountered in less_equal
  right = np.count_nonzero(a <= score)


,Ticker,Price,One-Year Price Return,Beta,One-Year IM,One-Year IM-Percentile,Six-Month Price Return,Six-Month IM,Six-Month IM-Percentile,Three-Month Price Return,Three-Month IM,Three-Month IM-Percentile,Piotroski Score,ROC,ROC Percentile,Number of Shares to Buy
0,A,168.87,0.591259,0.947536,0.275018,0.79604,0.329178,0.199607,0.885149,0.296226,0.225388,0.954455,N/A,N/A,N/A,N/A
1,AAL,19.30,0.500988,1.154142,0.115791,0.712871,0.002749,-0.155073,0.219802,-0.187446,-0.27373,0.045545,N/A,N/A,N/A,N/A
2,AAP,215.62,0.304037,0.714996,0.065406,0.465347,0.265575,0.167803,0.786139,0.078741,0.025287,0.669307,N/A,N/A,N/A,N/A
3,AAPL,148.81,0.252107,1.577257,-0.274305,0.390099,0.136210,-0.079472,0.510891,0.183090,0.065173,0.873267,N/A,N/A,N/A,N/A
4,ABBV,119.25,0.271548,0.686594,0.042396,0.419802,0.149568,0.05568,0.564356,0.029593,-0.021737,0.526733,N/A,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,135.16,0.291510,0.701742,0.057303,0.451485,0.283499,0.18754,0.811881,0.135743,0.08328,0.790099,N/A,N/A,N/A,N/A
501,ZBH,149.76,0.100811,0.967284,-0.222022,0.174257,-0.078482,-0.210752,0.093069,-0.132610,-0.204925,0.10495,N/A,N/A,N/A,N/A
502,ZBRA,591.28,0.884672,1.327935,0.441471,0.930693,0.152769,-0.028819,0.578218,0.167733,0.068455,0.843564,N/A,N/A,N/A,N/A
503,ZION,53.99,0.543313,1.275753,0.117529,0.754455,0.041602,-0.13285,0.312871,-0.070784,-0.16616,0.247525,N/A,N/A,N/A,N/A


# Step 4 - Advanced Buy Rules

Now that we have finished our momentum criterion we are going to have a look at the two additional (value-based) criteria we will use for stock selection.

These are:

- Stock not in bottom quartile in terms of Return on Capital (ROC)
- Stock's Piotroski-Score >= 3

Further information about these two metrics can be found in the following two articles:

- https://www.investopedia.com/ask/answers/09/return-on-equity-vs-return-on-capital.asp
- https://www.investopedia.com/terms/p/piotroski-score.asp

We are going to start by calculating the Piotroski Score for each stock.

### Piotroski-Score

Originally developed by Chicago Accounting Professor Joseph Piotroski, the _Piotroski-F-Score_ is a popular metric used to judge value stocks.

The nine aspects are based on accounting results over a number of years; a point is awarded each time a standard is met, resulting in an overall score.

The four profitability criteria are:

- Positive net income
- Positive return on assets in the current year
- Positive operating cash flow in the current year
- Cash flow from operations being greater than net income (= quality of earnings)

Leverage, liquidity, and source of funds criteria are:

- Lower ratio of long term debt in the current period, compared to the previous year (= decreased leverage)
- Higher current ratio this year compared to the previous year (= more liquidity)
- No new shares were issued in the last year (= lack of dilution)

Operating efficiency criteria include:

- Higher gross margin compared to the previous year
- Higher asset turnover ratio compared to the previous year

For this project we only want to consider stocks with a Piotroski score of 3 or higher.

The following code provides one way to calculate the score for each stock and add it to our dataframe. The _pyEX_ library is used in order to make more customisable API-Calls.

Please note that the following code has a rather lengthy runtime. I've added a print statement so the progress can be followed.

In [12]:
F_SCORE = []

for symbol_string in symbol_strings:
    for symbol in symbol_string.split(','):
        c = pyEX.Client(api_token = IEX_CLOUD_API_TOKEN, version = 'sandbox')
        # The data we want to pull from IEX
        income_statement = c.incomeStatement(symbol, 'annual', 2)
        balance_sheet = c.balanceSheet(symbol, 'annual', 2)
        adv_fundamentals = c.fundamentals(symbol, 'annual')
        cash_flow = c.cashFlow(symbol, 'annual')
        print(f"Analyzing {symbol}")

        try:
        ### 
        # Positive net income
        ###

            def net_income(income_statement):
                ni = income_statement[0]['netIncome']

                if ni > 0:
                    return 1
                else:
                    return 0

        ###
        # Positive return on assets in the current year
        ###

            def positive_roa(income_statement, balance_sheet):
                ni = income_statement[0]['netIncome']
                assets = balance_sheet[0]['totalAssets']

                if ni / assets > 0:
                    return 1
                else:
                    return 0

        ###
        # Positive operating cash flow in the current year
        ###

            def positive_ocf(adv_fundamentals):
                ocf = adv_fundamentals[0]['cashFlowOperating']

                if ocf > 0:
                    return 1
                else:
                    return 0

        ###
        # Cash flow from operations being greater than net income
        ###

            def quality_of_earnings(adv_fundamentals, income_statement):
                ocf = adv_fundamentals[0]['cashFlowOperating']
                ni = income_statement[0]['netIncome']

                if ocf > ni:
                    return 1
                else:
                    return 0

        ###
        # Lower ratio of long term debt in the current period, compared to the previous year
        ###

            def decreased_leverage(balance_sheet):
                debt_cur = balance_sheet[0]['longTermDebt']
                debt_prev = balance_sheet[1]['longTermDebt']

                if debt_cur < debt_prev:
                    return 1
                else:
                    return 0

        ###
        # Higher current ratio this year compared to the previous year
        ###

            def more_liquidity(balance_sheet):
                ratio_cur = balance_sheet[0]['currentAssets'] / balance_sheet[0]['totalCurrentLiabilities']
                ratio_prev = balance_sheet[1]['currentAssets'] / balance_sheet[1]['totalCurrentLiabilities']

                if ratio_cur > ratio_prev:
                    return 1
                else:
                    return 0

        ###
        # No new shares were issued in the last year
        # This category cannot be easily replicated with IEX
        # Instead we use negative cashflow-from-financing as a replacement
        ###

            def negative_cff(cash_flow):
                cff = cash_flow[0]['cashFlowFinancing']

                if cff < 0:
                    return 1
                else:
                    return 0

        ###
        # Higher gross margin compared to the previous year
        ###

            def higher_gm(income_statement):
                gm_cur = income_statement[0]['grossProfit'] / income_statement[0]['totalRevenue']
                gm_prev = income_statement[1]['grossProfit'] / income_statement[1]['totalRevenue']

                if gm_cur > gm_prev:
                    return 1
                else:
                    return 0

        ###
        # Higher asset turnover ratio compared to the previous year
        ###

            def asset_turnover_improving(income_statement, balance_sheet):
                at_cur = income_statement[0]['totalRevenue'] / balance_sheet[0]['totalAssets']
                at_prev = income_statement[1]['totalRevenue'] / balance_sheet[1]['totalAssets']

                if at_cur > at_prev:
                    return 1
                else:
                    return 0


            f_score = net_income(income_statement) + positive_roa(income_statement, balance_sheet) + positive_ocf(adv_fundamentals) + quality_of_earnings(adv_fundamentals, income_statement) + decreased_leverage(balance_sheet) + more_liquidity(balance_sheet) + negative_cff(cash_flow) + higher_gm(income_statement) + asset_turnover_improving(income_statement, balance_sheet)
            print(f'{symbol} f score is {f_score}')
            F_SCORE.append(f_score)

        except IndexError:
            print('Missing data!')
            F_SCORE.append(-1)

            
momentum_df['Piotroski Score'] = F_SCORE


Analyzing A
A f score is 7
Analyzing AAL
AAL f score is 2
Analyzing AAP
AAP f score is 7
Analyzing AAPL
AAPL f score is 6
Analyzing ABBV
ABBV f score is 5
Analyzing ABC
ABC f score is 5
Analyzing ABMD
ABMD f score is 6
Analyzing ABT
ABT f score is 7
Analyzing ACN
ACN f score is 7
Analyzing ADBE
ADBE f score is 7
Analyzing ADI
ADI f score is 7
Analyzing ADM
ADM f score is 4
Analyzing ADP
ADP f score is 5
Analyzing ADSK
ADSK f score is 7
Analyzing AEE
AEE f score is 6
Analyzing AEP
AEP f score is 6
Analyzing AES
AES f score is 6
Analyzing AFL
AFL f score is 5
Analyzing AIG
AIG f score is 2
Analyzing AIV
AIV f score is 5
Analyzing AIZ
AIZ f score is 7
Analyzing AJG
AJG f score is 7
Analyzing AKAM
AKAM f score is 5
Analyzing ALB
ALB f score is 5
Analyzing ALGN
ALGN f score is 5
Analyzing ALK
ALK f score is 4
Analyzing ALL
ALL f score is 6
Analyzing ALLE
ALLE f score is 8
Analyzing ALXN
ALXN f score is 6
Analyzing AMAT
AMAT f score is 7
Analyzing AMCR
AMCR f score is 8
Analyzing AMD
AMD f s

Analyzing KEYS
KEYS f score is 7
Analyzing KHC
KHC f score is 8
Analyzing KIM
KIM f score is 5
Analyzing KLAC
KLAC f score is 7
Analyzing KMB
KMB f score is 7
Analyzing KMI
KMI f score is 6
Analyzing KMX
KMX f score is 4
Analyzing KO
KO f score is 6
Analyzing KR
KR f score is 7
Analyzing KSS
KSS f score is 4
Analyzing KSU
KSU f score is 5
Analyzing L
L f score is 5
Analyzing LB
LB f score is 6
Analyzing LDOS
LDOS f score is 6
Analyzing LEG
LEG f score is 6
Analyzing LEN
LEN f score is 7
Analyzing LH
LH f score is 9
Analyzing LHX
LHX f score is 6
Analyzing LIN
LIN f score is 6
Analyzing LKQ
LKQ f score is 7
Analyzing LLY
LLY f score is 6
Analyzing LMT
LMT f score is 6
Analyzing LNC
LNC f score is 6
Analyzing LNT
LNT f score is 5
Analyzing LOW
LOW f score is 8
Analyzing LRCX
LRCX f score is 5
Analyzing LUV
LUV f score is 4
Analyzing LVS
LVS f score is 1
Analyzing LW
LW f score is 8
Analyzing LYB
LYB f score is 6
Analyzing LYV
LYV f score is 2
Analyzing MA
MA f score is 7
Analyzing MAA
MA

### Return on Capital

Return on Capital (ROC) is a well-known metric used by investors and institutions to decide between competing for investment options.

It measures a corporation's profitability in relation to stockholders’ equity while also taking into account the total amount of debt owed by the company. It can be calculated using the following formula:

\begin{equation}
\text{ROC} = \frac{\text{Net Income}}{\text{Debt} + \text{Equity}}
\end{equation}

In [13]:
ROC = []

for symbol_string in symbol_strings:
    for symbol in symbol_string.split(','):
        try:
            c = pyEX.Client(api_token = IEX_CLOUD_API_TOKEN, version = 'sandbox')
            i = c.incomeStatement(symbol, 'annual')
            b = c.balanceSheet(symbol, 'annual')

            net_income = i[0]['netIncome']
            debt = b[0]['longTermDebt']
            equity = b[0]['totalAssets'] - b[0]['totalLiabilities']

            roc = net_income / debt + equity
            roc = round(roc)
            ROC.append(roc)
            
        except (IndexError, ZeroDivisionError):
            ROC.append(-1)
        
        
momentum_df['ROC'] = ROC

Since this metric is used for comparing companies, rather than as an absolute value in itself, we calculate the ROC Percentiles like we did for Momentum in Step 3.

In [14]:
for row in momentum_df.index:
    change_col = 'ROC'
    percentile_col = 'ROC Percentile'
    if momentum_df.loc[row, change_col] == -1:
        momentum_df.loc[row, change_col] = 0.0

for row in momentum_df.index:       
    change_col = 'ROC'
    percentile_col = 'ROC Percentile'            
    momentum_df.loc[row, percentile_col] = score(momentum_df[change_col], momentum_df.loc[row, change_col]) / 100
        
momentum_df

,Ticker,Price,One-Year Price Return,Beta,One-Year IM,One-Year IM-Percentile,Six-Month Price Return,Six-Month IM,Six-Month IM-Percentile,Three-Month Price Return,Three-Month IM,Three-Month IM-Percentile,Piotroski Score,ROC,ROC Percentile,Number of Shares to Buy
0,A,168.87,0.591259,0.947536,0.275018,0.79604,0.329178,0.199607,0.885149,0.296226,0.225388,0.954455,7,5032582130,0.491089,N/A
1,AAL,19.30,0.500988,1.154142,0.115791,0.712871,0.002749,-0.155073,0.219802,-0.187446,-0.27373,0.045545,2,-6258531497,0.011881,N/A
2,AAP,215.62,0.304037,0.714996,0.065406,0.465347,0.265575,0.167803,0.786139,0.078741,0.025287,0.669307,7,3846826903,0.427723,N/A
3,AAPL,148.81,0.252107,1.577257,-0.274305,0.390099,0.136210,-0.079472,0.510891,0.183090,0.065173,0.873267,6,65857326487,0.974257,N/A
4,ABBV,119.25,0.271548,0.686594,0.042396,0.419802,0.149568,0.05568,0.564356,0.029593,-0.021737,0.526733,5,15079071899,0.80396,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,135.16,0.291510,0.701742,0.057303,0.451485,0.283499,0.18754,0.811881,0.135743,0.08328,0.790099,6,-7944159734,0.007921,N/A
501,ZBH,149.76,0.100811,0.967284,-0.222022,0.174257,-0.078482,-0.210752,0.093069,-0.132610,-0.204925,0.10495,4,12707696298,0.768317,N/A
502,ZBRA,591.28,0.884672,1.327935,0.441471,0.930693,0.152769,-0.028819,0.578218,0.167733,0.068455,0.843564,6,2132018323,0.29901,N/A
503,ZION,53.99,0.543313,1.275753,0.117529,0.754455,0.041602,-0.13285,0.312871,-0.070784,-0.16616,0.247525,4,0,0.115842,N/A


### Adding some colour

As you can tell, adding all this data to our dataframe does make it quite obscure.

We therefore add some colour coding to see if our conditions have been violated. As a quick reminder, we would like to:
- Select stocks from the highest two deciles of momentum stocks.
- Only consider those stocks that also rank in the upper half of stocks with highest quarterly and bi-quarterly idiosyncratic momentum.
- Select stocks with Piotroski-Score of 3 or higher.
- Exclude stocks that are in bottom quartile in terms of Return on Capital (ROC).

Here's how we can do that:

In [15]:
def color_quarterly_red(value):
    if value < 0.50:
        color = 'red'
    elif value >= 0.50:
        color = 'green'
    else:
        color = 'black'
    
    return 'color: %s' % color

def color_annual_red(value):
    if value < 0.80:
        color = 'red'
    elif value >= 0.80:
        color = 'green'
    else:
        color = 'black'
        
    return 'color: %s' % color

def piotroski_color(value):
    if value > 6:
        color = 'green'
    elif value in [3, 4, 5, 6]:
        color = 'orange'
    else: 
        color = 'red'
        
    return 'color: %s' % color

def roc_color(value):
    if value <= 0.25:
        color = 'red'
    else:
        color = 'green'
        
    return 'color: %s' % color
        

final_df_color = (momentum_df.style
                    .applymap(color_quarterly_red, subset=['Six-Month IM-Percentile', 'Three-Month IM-Percentile'])
                    .applymap(color_annual_red, subset=['One-Year IM-Percentile'])
                    .applymap(piotroski_color, subset=['Piotroski Score'])
                    .applymap(roc_color, subset=['ROC Percentile']))

final_df_color

,Ticker,Price,One-Year Price Return,Beta,One-Year IM,One-Year IM-Percentile,Six-Month Price Return,Six-Month IM,Six-Month IM-Percentile,Three-Month Price Return,Three-Month IM,Three-Month IM-Percentile,Piotroski Score,ROC,ROC Percentile,Number of Shares to Buy
0,A,168.870000,0.591259,0.947536,0.275018,0.796040,0.329178,0.199607,0.885149,0.296226,0.225388,0.954455,7,5032582130,0.491089,N/A
1,AAL,19.300000,0.500988,1.154142,0.115791,0.712871,0.002749,-0.155073,0.219802,-0.187446,-0.273730,0.045545,2,-6258531497,0.011881,N/A
2,AAP,215.620000,0.304037,0.714996,0.065406,0.465347,0.265575,0.167803,0.786139,0.078741,0.025287,0.669307,7,3846826903,0.427723,N/A
3,AAPL,148.810000,0.252107,1.577257,-0.274305,0.390099,0.136210,-0.079472,0.510891,0.183090,0.065173,0.873267,6,65857326487,0.974257,N/A
4,ABBV,119.250000,0.271548,0.686594,0.042396,0.419802,0.149568,0.055680,0.564356,0.029593,-0.021737,0.526733,5,15079071899,0.803960,N/A
5,ABC,122.250000,0.134092,0.632407,-0.076975,0.215842,0.141519,0.055041,0.532673,0.008399,-0.038880,0.445545,5,-841245262,0.033663,N/A
6,ABMD,363.170000,0.000180,0.911036,-0.303880,0.083168,0.004260,-0.120319,0.227723,0.218684,0.150574,0.910891,6,0,0.115842,N/A
7,ABT,130.120000,0.215989,0.655906,-0.002920,0.342574,0.030621,-0.059071,0.289109,0.087382,0.038346,0.693069,7,34046210486,0.916832,N/A
8,ACN,347.340000,0.359616,1.076605,0.000298,0.550495,0.311771,0.164551,0.857426,0.173093,0.092605,0.851485,7,16678638323,0.821782,N/A
9,ADBE,661.660000,0.319787,1.320017,-0.120771,0.481188,0.334048,0.153543,0.891089,0.334084,0.235399,0.964356,7,14086659724,0.784158,N/A


# Step 5 - Identifying stocks to buy

Now that we have visualised our conditions in our dataframe, it is time to filter our stocks based on our set conditions.

Here's how we can do that:

In [16]:
final_df1 = momentum_df[momentum_df['ROC Percentile'] >= 0.25]
final_df2 = final_df1[final_df1['Piotroski Score'] >= 3]
final_df3 = final_df2[final_df2['Three-Month IM-Percentile'] >= 0.50]
final_df4 = final_df3[final_df3['Six-Month IM-Percentile'] >= 0.50]
output_df = final_df4[final_df4['One-Year IM-Percentile'] >= 0.80]
output_df

,Ticker,Price,One-Year Price Return,Beta,One-Year IM,One-Year IM-Percentile,Six-Month Price Return,Six-Month IM,Six-Month IM-Percentile,Three-Month Price Return,Three-Month IM,Three-Month IM-Percentile,Piotroski Score,ROC,ROC Percentile,Number of Shares to Buy
23,ALB,229.41,1.136720,1.524301,0.627983,0.968317,0.385366,0.176926,0.934653,0.367860,0.253902,0.974257,5,4111769827,0.443564,N/A
78,CARR,57.54,0.792284,1.098679,0.425598,0.90099,0.477275,0.327036,0.976238,0.278054,0.195916,0.940594,7,6405991496,0.574257,N/A
82,CBRE,96.40,0.969205,1.19758,0.56951,0.950495,0.270461,0.106698,0.79604,0.083302,-0.00623,0.683168,7,7392120172,0.613861,N/A
86,CDW,197.59,0.627485,1.279319,0.20051,0.817822,0.216450,0.04151,0.710891,0.162501,0.066858,0.825743,7,1398969912,0.257426,N/A
106,COF,172.19,1.478765,1.151135,1.094572,0.986139,0.419338,0.261926,0.956436,0.077295,-0.008765,0.665347,5,61385055038,0.962376,N/A
162,EQR,86.27,0.596284,1.015783,0.257265,0.8,0.259822,0.120919,0.776238,0.134405,0.058464,0.782178,7,10928294788,0.714851,N/A
164,ESS,323.39,0.597851,0.927285,0.288369,0.80396,0.225666,0.098865,0.732673,0.129250,0.059926,0.772277,7,6050366895,0.550495,N/A
173,EXR,179.40,0.751772,0.397581,0.619079,0.879208,0.507051,0.452684,0.982178,0.229051,0.199328,0.924752,6,2662117835,0.324752,N/A
205,GOOGL,2786.18,0.698406,1.329807,0.254581,0.857426,0.310320,0.128476,0.853465,0.196574,0.097157,0.883168,5,221744928044,1.0,N/A
216,HCA,257.56,0.756358,0.949369,0.439505,0.881188,0.397842,0.268021,0.940594,0.218051,0.147076,0.908911,7,2802069971,0.338614,N/A


As we can see there were 30 stocks that met our conditions in this example. In line with our strategy we are going to pick the twenty stocks with highest One-Year Idiosyncratic Momentum.

In [17]:
output_df.sort_values('One-Year IM', ascending = False, inplace = True)
output_df = output_df[:20]
output_df.reset_index(drop = True, inplace = True)
output_df

C:\Users\Max\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Ticker,Price,One-Year Price Return,Beta,One-Year IM,One-Year IM-Percentile,Six-Month Price Return,Six-Month IM,Six-Month IM-Percentile,Three-Month Price Return,Three-Month IM,Three-Month IM-Percentile,Piotroski Score,ROC,ROC Percentile,Number of Shares to Buy
0,COF,172.19,1.478765,1.151135,1.094572,0.986139,0.419338,0.261926,0.956436,0.077295,-0.008765,0.665347,5,61385055038,0.962376,N/A
1,NUE,120.06,1.350708,1.146756,0.967976,0.980198,1.057001,0.900188,0.992079,0.167310,0.081578,0.841584,5,11786977192,0.744554,N/A
2,IT,308.90,1.197552,1.118255,0.824332,0.974257,0.649674,0.496758,0.986139,0.347623,0.264021,0.968317,8,1291773727,0.251485,N/A
3,TGT,256.00,0.658440,0,0.65844,0.829703,0.327393,0.327393,0.881188,0.140342,0.140342,0.79604,8,15687830364,0.813861,N/A
4,ALB,229.41,1.136720,1.524301,0.627983,0.968317,0.385366,0.176926,0.934653,0.367860,0.253902,0.974257,5,4111769827,0.443564,N/A
5,EXR,179.40,0.751772,0.397581,0.619079,0.879208,0.507051,0.452684,0.982178,0.229051,0.199328,0.924752,6,2662117835,0.324752,N/A
6,SPG,130.93,1.065037,1.380122,0.604419,0.958416,0.207192,0.018467,0.689109,0.063284,-0.039895,0.623762,6,3522532088,0.407921,N/A
7,CBRE,96.40,0.969205,1.19758,0.56951,0.950495,0.270461,0.106698,0.79604,0.083302,-0.00623,0.683168,7,7392120172,0.613861,N/A
8,PSA,324.73,0.671698,0.347973,0.555562,0.843564,0.402177,0.354594,0.948515,0.175644,0.14963,0.859406,6,8853547626,0.647525,N/A
9,JCI,73.60,0.797380,0.829229,0.520624,0.90495,0.318842,0.205449,0.869307,0.163725,0.101731,0.833663,4,17754347735,0.837624,N/A


# Step 6 - Calculating the Number of Shares to Buy

We still haven't calculated the number of shares of each stock to buy. We would like to create an equal-weight portfolio of any size. We use an input to let the user specify his portfolio size.

Since the API returns its data in US-Dollars, you may need to convert it for use with another currency. The following code automatically converts your input from € to $.

In [21]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio (in €): ')
    
    try:
        float(portfolio_size)
    except ValueError:
        print('That is not a number. \nPlease enter a float number: ')
        portfolio_size = input('Enter the size of your portfolio: ')

portfolio_input()
cr = CurrencyRates()
rate_euro_in_dollar = cr.convert('EUR', 'USD', 1)
portfolio_size = float(portfolio_size) * float(rate_euro_in_dollar)
print(f'Your portfolio size is ${portfolio_size}.')

Enter the size of your portfolio (in €): 20000
Your portfolio size is $23342.0.


We also need to round down the result since we neither want to end up with fractional shares nor go over the specified portfolio size.

In [22]:
position_size = float(portfolio_size) / len(output_df.index)
for i in output_df.index:
    output_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / output_df['Price'][i])
    
output_df

C:\Users\Max\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,One-Year Price Return,Beta,One-Year IM,One-Year IM-Percentile,Six-Month Price Return,Six-Month IM,Six-Month IM-Percentile,Three-Month Price Return,Three-Month IM,Three-Month IM-Percentile,Piotroski Score,ROC,ROC Percentile,Number of Shares to Buy
0,COF,172.19,1.478765,1.151135,1.094572,0.986139,0.419338,0.261926,0.956436,0.077295,-0.008765,0.665347,5,61385055038,0.962376,6
1,NUE,120.06,1.350708,1.146756,0.967976,0.980198,1.057001,0.900188,0.992079,0.167310,0.081578,0.841584,5,11786977192,0.744554,9
2,IT,308.90,1.197552,1.118255,0.824332,0.974257,0.649674,0.496758,0.986139,0.347623,0.264021,0.968317,8,1291773727,0.251485,3
3,TGT,256.00,0.658440,0,0.65844,0.829703,0.327393,0.327393,0.881188,0.140342,0.140342,0.79604,8,15687830364,0.813861,4
4,ALB,229.41,1.136720,1.524301,0.627983,0.968317,0.385366,0.176926,0.934653,0.367860,0.253902,0.974257,5,4111769827,0.443564,5
5,EXR,179.40,0.751772,0.397581,0.619079,0.879208,0.507051,0.452684,0.982178,0.229051,0.199328,0.924752,6,2662117835,0.324752,6
6,SPG,130.93,1.065037,1.380122,0.604419,0.958416,0.207192,0.018467,0.689109,0.063284,-0.039895,0.623762,6,3522532088,0.407921,8
7,CBRE,96.40,0.969205,1.19758,0.56951,0.950495,0.270461,0.106698,0.79604,0.083302,-0.00623,0.683168,7,7392120172,0.613861,12
8,PSA,324.73,0.671698,0.347973,0.555562,0.843564,0.402177,0.354594,0.948515,0.175644,0.14963,0.859406,6,8853547626,0.647525,3
9,JCI,73.60,0.797380,0.829229,0.520624,0.90495,0.318842,0.205449,0.869307,0.163725,0.101731,0.833663,4,17754347735,0.837624,15


Finally, we would like to know how much money we will have left following our trades:

In [23]:
for i in output_df.index:
    spent = output_df['Price'] * output_df['Number of Shares to Buy']

rate_dollar_in_euro = cr.convert('USD', 'EUR', 1)
total_spent = spent.sum()
total_spent = round(total_spent * float(rate_dollar_in_euro))
balance = float(portfolio_size) * float(rate_dollar_in_euro) - total_spent
balance = round(balance)
print(f'Following the trades, portfolio size will be €{total_spent} and the remaining balance will be €{balance}.')

Following the trades, portfolio size will be €17881 and the remaining balance will be €2119.


# Step 7 - Saving our Results and Formatting our Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. 

We will also save the file as a pickle-object for further use.

### Initializing our XlsxWriter Object

In [24]:
writer = pd.ExcelWriter('momentum_strategy_test.xlsx', engine = 'xlsxwriter')
output_df.to_excel(writer, sheet_name = "Momentum Strategy", index = False)

### Creating the Formats

In [25]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

### Applying the formats and saving the output

In [26]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template], 
    'C': ['One-Year Price Return', percent_template],
    'D': ['Beta', float_template],
    'E': ['One-Year IM', percent_template], 
    'F': ['One-Year IM-Percentile', percent_template], 
    'G': ['Six-Month Price Return', percent_template], 
    'H': ['Six-Month IM', percent_template], 
    'I': ['Six-Month IM-Percentile', percent_template],
    'J': ['Three-Month Price Return', percent_template], 
    'K': ['Three-Month IM', percent_template], 
    'L': ['Three-Month IM-Percentile', percent_template], 
    'M': ['Piotroski Score', integer_template], 
    'N': ['ROC', dollar_template],
    'O': ['ROC Percentile', percent_template], 
    'P': ['Number of Shares to Buy', integer_template]                 
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1]) # 25 = column width
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
    
writer.save()

### Save as pickle object

Since we might want to use this output for further analysis, it may be better to save the data in a more accessible file format. Pickle provides a good and easy-to-use option for that.

In [27]:
pickle.dump(output_df, open('momentum_strategy_test.pkl', 'wb'))